# Assignment 2

![](https://media.giphy.com/media/xT9C25UNTwfZuk85WP/giphy-downsized-large.gif)

Remember the rules of ~Fight~ Code Club:
1. ALWAYS DOCUMENT
2. Cite resources that you use (paste links)
3. Include the names people who you worked with
4. Be neat and organized

## Scrape and Clean data

Based on you proposal, scrape or collect your data:

1. One variable must be either: (40 pts)
    1. scraped from the web OR;
    2. collected from an API AND you must create one new variable that is "new" to the best of your knowledge (combination of other variables representing something new).

2. You must have at least 3 variables, but you may include as many as you want into you final dataset. Likely, you will want to include more to make graphs and regressions. (30pts)

3. You must *be able* to run a regression that makes some sense with this data (the regression doesn't have to be a complete model). Briefly describe one regression you would run with your variables. (**DO NOT** run a regression, yet). (15pts)

4. You must have one combined and cleaned dataset (15 pts)

You must submit one python notebook on how you scraped/gathered data from an api, and how you combined and cleaned you data. I should be able to run your code and reproduce your final data set.  

The other variables that you choose to include do not have to be collected by API or webscraped, but you do have to combine the files and clean the dataset with python.

Thus, you must submit:
- Your finalized data set (only one) (note: you may add more variables in the future).
- Your documented python notebook
- Any associated data files needed to produce the final dataset.

You will be evaluated on:
- Completeness of the data
- Quality of the code 
- The creativity of the new variable/webscraped data you gather

Be sure to upload ALL associated files for your code to run. I will run your code from beginning to end - make it easy for me to replicate your code.

## Data collection process

### Variables 
<span style="background-color: #FFFF00">Done</span>

1.Outputs and employment from different parts of the manufacturing sector(maybe)

2.CPI - Urban and All Not Including Food and Energy <span style="background-color: #FFFF00">Done</span>

3.National Income Level <span style="background-color: #FFFF00">Done</span>

4.Demand for raw materials(Baltic Dry Index)

5.General Monthly Employment numbers and monthly GDP across at least 20 years <span style="background-color: #FFFF00">Done</span>

6.Cost of capital

7.Measure for investment demand for manufacturing businesses

8.Monetary Policy shifts expectations and reality

9.Technology Index that controls for the increase of innovation in the economy

10.Michigan State’s Consumer Price Index <span style="background-color: #FFFF00">Done</span>

[Merging help](https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns)

Considering using this: [NBER Manufacturing Database](https://www.nber.org/research/data/nber-ces-manufacturing-industry-database)


### Utilizing Fred API
[Fred API Documentation](https://fred.stlouisfed.org/docs/api/fred/)

API KEY: 565e55b6fbd720965afae15454629fae

In [1]:
#Packages needed
import requests
import json
import pandas as pd
from pandas.tseries.offsets import DateOffset
from datetime import date

In [2]:
#makes request to connect to api for Federal Funds Rate series
res = requests.get("https://api.stlouisfed.org/fred/series/observations?series_id=DFF&api_key=565e55b6fbd720965afae15454629fae&file_type=json")
json_ffr = res.json()

In [3]:
#creating dictionary to put data 
dt_fr = {}
dt_fr["dates"] = []
dt_fr["ff_rate"] = []

#loops through observations and appends observed days to the dataframe
for y in range(0, len(json_ffr["observations"])):
    dt_fr["dates"].append(json_ffr["observations"][y].get('date'))
    dt_fr["ff_rate"].append(json_ffr["observations"][y].get('value'))

#Converts dictionary to dataframe and making dates into datetime type and rates into numeric
dt_fr = pd.DataFrame(dt_fr)
dt_fr["dates"] = pd.to_datetime(dt_fr["dates"])
dt_fr["ff_rate"] = pd.to_numeric(dt_fr["ff_rate"], errors = "coerce")

#subsets data and resets the index
dt_fr = dt_fr[dt_fr["dates"] >= "1998-01-01"].reset_index(drop = True, inplace = False)

In [4]:
#sets date as index to allow groupby of rates my month
dt_fr_avg = dt_fr.set_index(dt_fr["dates"]).groupby(pd.Grouper(freq = "M")).mean()

#Shifts the dates by one day to make it easier to merge and reset index
#shifting makes January 31st 2023 into February 1st 2023 instead
dt_fr_avg = dt_fr_avg.shift(freq = '1d')[["ff_rate"]].reset_index(0)
dt_fr_avg

,dates,ff_rate
0,1998-02-01,5.557419
1,1998-03-01,5.505714
2,1998-04-01,5.493548
3,1998-05-01,5.445333
4,1998-06-01,5.488710
...,...,...
311,2024-01-01,5.330000
312,2024-02-01,5.330000
313,2024-03-01,5.330000
314,2024-04-01,5.330000


### Using BLS API 
[BLS API Documentation](https://www.bls.gov/developers/api_signature_v2.htm#parameters)

API KEY: ee2f076f1d254305bc09f42aa498afab

In [5]:
#Pulls all the surveys that BLS has on hand
res = requests.get("https://api.bls.gov/publicAPI/v2/surveys")
json_survey = res.json()
json_survey

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 29,
 'message': [],
 'Results': {'survey': [{'survey_abbreviation': 'AP',
    'survey_name': 'Consumer Price Index - Average Price Data'},
   {'survey_abbreviation': 'BD',
    'survey_name': 'Business Employment Dynamics'},
   {'survey_abbreviation': 'BG',
    'survey_name': 'Collective Bargaining Agreements-State and Local Government'},
   {'survey_abbreviation': 'BP',
    'survey_name': 'Collective Bargaining Agreements-Private Sector'},
   {'survey_abbreviation': 'CB',
    'survey_name': 'Biennial Nonfatal Case and Demographic numbers and rates: selected characteristics'},
   {'survey_abbreviation': 'CC',
    'survey_name': 'Employer Costs for Employee Compensation'},
   {'survey_abbreviation': 'CD',
    'survey_name': 'Nonfatal cases involving days away from work: selected characteristics'},
   {'survey_abbreviation': 'CE',
    'survey_name': 'Employment, Hours, and Earnings from the Current Employment Statistics survey (National)'},

In [6]:
#Function works to merge the data tables by separating them from the dataframe it is in
#recieves the dictionary from the function called bls_json
def bls_wrangle(value):
    #makes list of columns that help differentiate the values after separating the data by series
    columns = ["man_emp","core_cpi","employment", "unemp_lvl"]
    
    #converts dates to datetime and all values to numeric
    value["dates"] = pd.to_datetime(value["dates"])
    value["value"] = pd.to_numeric(value["value"], errors = "coerce")
      
    #divide the dataframe into four dataframes that would be combine together later
    #the dataframe is divided by series and value column is renamed to make unique columns 
    df1 = value[value["series"] == series_id[0]][["dates", "value"]].rename(columns = {"value":columns[0]})
    df2 = value[value["series"] == series_id[1]][["dates", "value"]].rename(columns = {"value":columns[1]})
    df3 = value[value["series"] == series_id[2]][["dates", "value"]].rename(columns = {"value":columns[2]})
    df4 = value[value["series"] == series_id[3]][["dates", "value"]].rename(columns = {"value":columns[3]})
    
    #list of the frames
    frames = [df1, df2, df3, df4]
        
    #Merge on the first 2 frames and then merging sequentially through list of frames
    frame = pd.merge(df1, df2, on = "dates")
    for i in range(2, 4):
        frame = pd.merge(frame, frames[i])    
        frame = frame.sort_values(by = ["dates"])
    
    return(pd.DataFrame(frame))

                                    
#Places values into a dictionary with no sorting
def bls_json(json_l):
    #new dictionary 
    d_t = {}
    d_t["dates"] = []
    d_t["series"] = []
    d_t["value"] = []
    
    for i in range(0, len(json_l['Results']['series'])):
        serie = str(json_l['Results']['series'][i].get("seriesID"))
        data = json_l['Results']['series'][i].get("data") 
        for items in data:
            d_t["dates"].append((items.get("periodName") + " 01 " + items.get("year")))
            d_t["value"].append(items.get("value"))
            d_t["series"].append(serie)

    d_t = bls_wrangle(pd.DataFrame(d_t))
    return(d_t)

#Found these codes through the BLS Datafinder function
#CES3000000001 - Seasonally Adjusted - Employed and Office of Employment and Unemployment Statistics: Manfacturing - manemp
#CUSR0000SA0 - Seasonally Adjusted Consumer Price Index for All Urban Consumers, not including food and energy - cpi_u
#CES0000000001 - Seasonally Adjusted - Nonfarm Employment - employment_sa
#LNS13000000 - Seasonally Adjusted - Unemployment level in the thousands - unemp_lvl
series_id = ['CES3000000001', 'CUSR0000SA0L1E', 'CES0000000001', 'LNS13000000']
headers = {'Content-type' : 'application/json'}

#designates series and the range of the data and intiates the pull
data = json.dumps({"seriesid" : series_id, 
                   "startyear": 1998, "endyear" : 2011, 
                   "registrationkey": "ee2f076f1d254305bc09f42aa498afab"})
res = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data = data, headers = headers)
json_res = json.loads(res.text)

#gets value from the function which is a complete dataframe that has a time range between 1998-2011 
left = bls_json(json_res).reset_index(0)

#designates series and the range of the data and intiates the pull
data = json.dumps({"seriesid": series_id,
                   "startyear": 2012, "endyear": 2024,
                   "registrationkey": "ee2f076f1d254305bc09f42aa498afab"})
res = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data = data, headers = headers)
json_res = json.loads(res.text)

gets value from the function which is a complete dataframe that has a time range between 1998-2011
right = bls_json(json_res).reset_index(0)

#merges the right and left, and then resets the index while subsetting on the columns
bls_data = right.merge(left, how = "outer")
bls_data= bls_data.sort_values(by = "dates").reset_index(0)[["dates", "man_emp", "core_cpi", "employment","unemp_lvl"]]

### Using BEA API
[BEA API Documentation](https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf)

API KEY:  A2D2AB50-A251-4378-8CC1-95E51C78615E

In [7]:
#Gives the personal income dataset
res = requests.get("https://apps.bea.gov/api/data?&UserID=A2D2AB50-A251-4378-8CC1-95E51C78615E&method=GetData&DataSetName=NIPA&TableName=T20600&tableID=ALL&Frequency=M&Year=ALL&ResultFormat=JSON")
personal_income = res.json()

In [16]:
#Makes the dictionary the data would be put into
pi_bea = {}
pi_bea["dates"] = []
pi_bea["personal_income"] = []
pi_bea["days"] = []

#Looking through the dictionary to find the values of personal income
for l in range(0, len(personal_income["BEAAPI"]["Results"]["Data"])):
    if(personal_income["BEAAPI"]["Results"]["Data"][l].get("LineDescription") == "Personal income"):
        pi_bea["dates"].append(personal_income["BEAAPI"]["Results"]["Data"][l].get("TimePeriod"))
        pi_bea["personal_income"].append(personal_income["BEAAPI"]["Results"]["Data"][l].get("DataValue"))
        pi_bea["days"].append("01")
    
#Converts dictionary into dataframe
pi_bea = pd.DataFrame(pi_bea)

#Since the datetime function requires a specific type of format for the date I would have to append a series of ones 
#that would concatenated to creat the dates column and the dates column can be converted
pi_bea["dates"] = pi_bea["dates"].str[:4] + "-" + pi_bea["dates"].str[5:] + "-" + pi_bea["days"]
pi_bea["dates"] = pd.to_datetime(pi_bea["dates"])

#Subsets the data by dates and columns while also resetting the index
pi_bea = pi_bea[pi_bea["dates"] > "1997-12-01"].reset_index(drop = True)[["dates","personal_income"]]
pi_bea

,dates,personal_income
0,1998-01-01,"7,368,855"
1,1998-02-01,"7,416,483"
2,1998-03-01,"7,464,033"
3,1998-04-01,"7,501,729"
4,1998-05-01,"7,547,019"
...,...,...
309,2023-10-01,"23,230,949"
310,2023-11-01,"23,319,690"
311,2023-12-01,"23,396,977"
312,2024-01-01,"23,627,877"


### Census Bureau 
[Census Bureau API Documentation](https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf)

[Census Bureau API Documentation actual stuff](https://www2.census.gov/data/api-documentation/EITS_API_User_Guide_Dec2020.pdf)

API Key: a9c6bd12867708842174babc90f20cfaa20255a7

In [9]:
#Pulls all of the data from the M3 survey 
res = requests.get("https://api.census.gov/data/timeseries/eits/m3.json?get=cell_value,data_type_code,time_slot_id,seasonally_adj,category_code,error_data&for=US&time=from+1992-01+to+2024-03&key=a9c6bd12867708842174babc90f20cfaa20255a7")
data = res.json()

In [10]:
#Converts data ovject into dataframe directly because the values were placed in a list of lists
cbs = pd.DataFrame(data, columns = data[0]) 

#Subsets the data to get values that contribute to total manufacturing
cbs = cbs[cbs["category_code"] == "MTM"]

#Since the datetime function requires a specific type of format for the date I would have to append a series of ones 
#that would concatenated to creat the dates column and the dates column can be converted
cbs["dates"] = cbs["time"] + "-01"    
cbs["dates"] = pd.to_datetime(cbs["dates"])

In [11]:
#Subsets data by specific data types: Value of Shipments(VS), Finished Inventory(FI), Total Inventory(TI)
#Also made sure the data was seasonally adjusted
cbs = cbs[((cbs["data_type_code"] == "VS") | (cbs["data_type_code"] == "FI") | (cbs["data_type_code"] == "TI")) & (cbs["seasonally_adj"] == "yes")]

#subsets the columns to make it easier when combining with later tables
cbs = cbs[["dates", "cell_value", "data_type_code", "seasonally_adj", "category_code"]].reset_index(drop = True, inplace = False )

In [12]:
#Calls the csv through the path name
consumer = pd.read_csv("C:/Users/marvi/Downloads/consumer.csv")
consumer["dates"] = pd.to_datetime(consumer["dates"])
consumer = consumer.set_index(consumer["dates"]).shift(freq = '1d')[["consumer_confidence"]].reset_index(0)

In [13]:
#Calls the execel sheet through its path name
gdp = pd.read_excel("C:/Users/marvi/OneDrive/Documents/GitHub/Stock_Markets/excel_sheets/monthly_gdp.xlsx")
gdp["dates"] = pd.to_datetime(gdp["dates"])
gdp = gdp[["dates", "Nominal_GDP_Index"]]

In [14]:
#List of the Dataframes created in this notebook
list_frames = [gdp, consumer, bls_data, pi_bea, dt_fr_avg]

#List of the Column names that would be in Census Bureau dataset
list_cbs_cl = ["VS", "FI", "TI"]

#iterates through the list of dataframes and joins them sequentially
for i in range(0, len(list_frames)):
    if i == 0:
        continue
    elif i == 1:
        df1 = pd.merge(list_frames[i], list_frames[i-1])
    elif i > 1 & i <= 4:  
        df1 = pd.merge(df1, list_frames[i])

#Iterates through the columns to create a new dataframe from somewhat cleaned sensus buearu dataframe from earlier
#After creating the new dataframe it would have its column renamed and then joined with the dataframe above
for j in range(0, len(list_cbs_cl)):
    df2 = cbs[(cbs["data_type_code"] == list_cbs_cl[j]) & (cbs["category_code"] == "MTM") & (cbs["seasonally_adj"] == "yes")]
    df2 = df2[["dates", "cell_value"]].reset_index(drop = True, inplace = False).rename(columns = {"cell_value":list_cbs_cl[j]})
    df1 = pd.merge(df1, df2, on = "dates")
df1


,dates,consumer_confidence,Nominal_GDP_Index,man_emp,core_cpi,employment,unemp_lvl,personal_income,days,ff_rate,VS,FI,TI
0,1998-02-01,106.6,8772.700661,17627.0,171.900,125018.0,6306.0,"7,416,483",01,5.557419,329617,153489,447466
1,1998-03-01,110.4,8894.547995,17637.0,172.200,125164.0,6422.0,"7,464,033",01,5.505714,328480,153966,448486
2,1998-04-01,106.5,8932.191344,17637.0,172.500,125445.0,5941.0,"7,501,729",01,5.493548,324127,153449,449663
3,1998-05-01,108.7,8925.512418,17624.0,172.900,125846.0,6047.0,"7,547,019",01,5.445333,326081,153687,450268
4,1998-06-01,106.5,8953.129642,17608.0,173.200,126077.0,6212.0,"7,584,421",01,5.488710,318990,153379,451186
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,2023-08-01,71.5,27386.299100,12941.0,309.656,156421.0,6340.0,"23,094,503",01,5.120000,585976,298064,855320
307,2023-09-01,69.4,27624.369911,12954.0,310.644,156667.0,6347.0,"23,176,244",01,5.330000,585918,298548,856463
308,2023-10-01,67.8,27828.287502,12923.0,311.390,156832.0,6443.0,"23,230,949",01,5.330000,578039,298530,856465
309,2023-11-01,63.8,27742.860313,12948.0,312.349,157014.0,6262.0,"23,319,690",01,5.330000,580730,298333,856730


## Potential Regression(s)

##### y(GDP, Employment Level)<sub>t</sub> = GDP<sub>t-k</sub> + Employment<sub>t-k</sub> 
##### y(Value of Shipments, Employment Level)<sub>t</sub> = Value of Shipments<sub>t-k</sub> + Manufacturing Employment<sub>t-k</sub>
Based off the law made by Okun, I will working off the assumption that a change in employment would induce a change in output 

- These would be Vectorautoregressions(VAR) that depicts relationship between output and employment. I will be using Gross Domestic Product and both Employment Levels and Unemployment Levels and that will help detemine if the relationship can be seen in the general economy. Then there will also be a model for specifically the manufacturing industry, that would make use of value of shipments as the measure of output and manufacturing employment levels
- Utilizes monthly data of employment and inflation adjusted GDP values
- After running the VAR, I will use a granger causality test to determine the predictiveness of both variables and their respective lags 
- Following the granger test, I will make use of a impulse response function to determine the effects over multiple periods


##### Value of Shipments<sub>t</sub> = Manufacturing Employment<sub>t</sub> + Personal Income<sub>t</sub> + Baltic Dry Index<sub>t</sub> + Federal Funds Rate<sub>t</sub>
This is a time series regressions that predicts how the value of shipments change. Based off the results from the VAR and the underlying tests, I would determine if a lag in manufacturing employment affects the value of shipments at time *t*. One assumption I make is that as factory owners recognize that in maximizing output there would need to be an increase in labor. That would potentially hurt the objective of maximizing profit, so the firm may purchase technology that would produce marginally lower expenses than an employee and increase potential output. Additionally, a decrease in national personal income may affect the value of shipments as more people save rather than consume. Shifts in the demand for the materials to produce would affect the values of shipments as well considering that the industry may need those inputs. Finally the movements and expectations around monetary policy would shift the movements of the market and potentially affect the worth of those shipments     

